In [ ]:
# Select the varaible based on your environment
# On cloud
metadata_table_path = '/mnt/customer360-blob-output/C360/UTILITIES/metadata_table'
# On premise
# metadata_table_path = '/projects/prod/c360/data/UTILITIES/metadata_table'

In [1]:
#Before running this notebook, make sure to create a backup of metadata table.

# import datetime
# now = datetime.datetime.now()
# bkp_path = metadata_table_path+now.strftime("%Y_%m_%d_%H_%M_%S")+'/metadata_table_bkp/'
#dbutils.fs.cp(metadata_table_path, bkp_path)

In [2]:
####################### Daily level (L1 and  L1-L4 layer) datasets #######################
# In this scenario, we need to put the reset data as (corrupt_date - 1) day, where corrupt_date is the minimum of date(s) for which corrupt data is loaded.

# For e.g
# Let's say, for dataset - D1, we have found out that corrupt/dirty data got loaded from 24th April (24/04/2020) till 26th April (26/04/2020).
# Then the "reset_date_for_dataset" will be '23rd April' (23/04/2020)

# Because this is the date till which correct data had already been processed.



####################### Weekly level (L2 and L2-L4 layer) datasets #######################
# In this scenario, we need to put the reset data as "start of previous week", where corrupt_date is the minimum of week(s) for which corrupt data is loaded.

# For e.g
# Let's say, for dataset - D2, we have found out that corrupt/dirty data got loaded for week of 20th April (20/04/2020) and 27th April (27/04/2020).
# Then the "reset_date_for_dataset" will be '13rd April' (13/04/2020)

# Because this is the week till which correct data had already been processed.




####################### Monthly level (L3 and L3-L4 layer) datasets #######################
# In this scenario, we need to put the reset data as "start of previous month", where corrupt_date is the minimum of months(s) for which corrupt data is loaded.

# For e.g
# Let's say, for dataset - D3, we have found out that corrupt/dirty data got loaded from 01st April (01/04/2020) and 01st May (01/05/2020).
# Then the "reset_date_for_dataset" will be '01st Mar' (01/03/2020)

# Because this is the month till which correct data had already been processed.

In [3]:
dataset_name_for_reset = 'test'
dataset_path_for_reset = '/mnt/customer360-blob-output/C360/REVENUE/l4_features/test/'
reset_date_for_dataset = '1970-01-01'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [5]:
def _get_spark():
  spark = SparkSession.builder.appName("project_customer_360_data_flow_control").getOrCreate()
  spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
  spark.conf.set("spark.sql.parquet.binaryAsString", "true")
  spark.conf.set("spark.sql.shuffle.partitions", 200)
  spark.conf.set("spark.sql.files.maxPartitionBytes", 1024*1024*256)
  return spark
spark = _get_spark()
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [6]:
mtdt_tbl = spark.read.parquet(metadata_table_path)

In [7]:
metadata_table_update_df = spark.range(1)

metadata_table_update_df = (
            metadata_table_update_df.withColumn("table_name", F.lit(dataset_name_for_reset))
                .withColumn("table_path", F.lit(dataset_path_for_reset))
                .withColumn("write_mode", F.lit("overwrite"))
                .withColumn("target_max_data_load_date", F.to_date(F.lit(reset_date_for_dataset), "yyyy-MM-dd"))
                .withColumn("updated_on", F.current_date())
                .drop("id")
                    )

metadata_table_update_df.write.partitionBy("table_name").format("parquet").mode("overwrite").save(metadata_table_path)